In [1]:
import os
import yaml
import urllib
from PIL import Image
from enum import Enum
from pycocotools.coco import COCO

import xml.etree.cElementTree as ET
import glob
import argparse
import numpy as np
import json
import numpy
import cv2
from collections import OrderedDict
import scipy.misc
from skimage import measure   
from shapely.geometry import Polygon, MultiPolygon, MultiPoint
import random
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import shutil

WORKING_DIR = '/media/dean/datastore/datasets/BerkeleyDeepDrive/scalabel/'
#IMAGE_LIST_DIR = os.path.join(WORKING_DIR, 'examples/image_list.yml')
IMAGE_LIST_DIR = '/media/dean/datastore/datasets/BerkeleyDeepDrive/bdd100k/images/100k/train/image_list.yml'
#LABEL_LIST_DIR = os.path.join(WORKING_DIR, 'examples/scalabel_results.json')
LABEL_LIST_DIR = '/media/dean/datastore/datasets/BerkeleyDeepDrive/bdd100k/labels/100k/train/'
COCO_DIRECTORY = os.path.join(WORKING_DIR, 'data/coco')
DATACACHE = os.path.join(COCO_DIRECTORY, 'images/train2014')
img_prefix = 'COCO_train2014_0000'
DEFAULT_IMG_EXTENSION = '.jpg'

FIXED_COCO_ANNOTATIONS_FILE = os.path.join(COCO_DIRECTORY,'annotations/fixed_instances_train2014.json')
SCALABEL_COCO_ANNOTATIONS_FILE = os.path.join(COCO_DIRECTORY,'annotations/scalabel_instances_train2014.json')

In [2]:
def maybe_download(source_url, filename):
    os.makedirs(DATACACHE, exist_ok = True)
    filepath = os.path.join(DATACACHE, filename)
    if os.path.exists(source_url) and not os.path.exists(filepath):
        # Copy image into training directory
        shutil.copyfile(source_url, filepath)
    elif not os.path.exists(filepath):
        filepath, _ = urllib.request.urlretrieve(source_url, filepath)
        statinfo = os.stat(filepath)
        #print('Succesfully downloaded:', filepath, '| % d MB.\n' % int(statinfo.st_size*1e-6))
    return filepath

In [3]:
class Format(Enum):
    scalabel = 0
    coco = 1
    darknet = 2
    bdd = 3

In [4]:
class Dataset(object):
    def __init__(self, image_list, label_list, data_format=Format.scalabel, output_path=WORKING_DIR):
        self._images = {}
        self._annotations = {}
        
        if data_format == Format.scalabel:
            with open(image_list, 'r') as stream:
                image_data = yaml.load(stream)
                if image_data:
                    for img in image_data:
                        img_url = img['url']
                        fname = os.path.split(img_url)[-1]
                        full_path = maybe_download(img_url, img_prefix+fname)
                        im = Image.open(full_path)
                        width, height = im.size
                        self._images[img_prefix+fname] = {'url': img_url, 'coco_path': full_path,
                                             'width': width, 'height': height}
        
            
            # Import Labels            
            with open(label_list, 'r') as f:
                data = json.load(f)
                
                for ann in data:
                    fname = os.path.split(ann['url'])[-1]
                    self._annotations[img_prefix+fname] = ann['labels']
                    
                    
        elif data_format == Format.bdd:
            with open(image_list, 'r') as stream:
                image_data = yaml.load(stream)
                if image_data:
                    for img in image_data:
                        img_url = img['url']
                        fname = os.path.split(img_url)[-1]
                        full_path = maybe_download(img_url, img_prefix+fname)
                        
                        im = Image.open(full_path)
                        width, height = im.size
                        self._images[img_prefix+fname] = {'url': img_url, 'coco_path': full_path,
                                             'width': width, 'height': height}
            
            
            # Get labels
            img_labels = glob.glob(os.path.join(label_list, '*.json'))
            print(len(img_labels))
            for i, img_label in enumerate(img_labels):
                with open(img_label, 'r') as f:
                    data = json.load(f)
                    fname = data['name']
                    if not fname.endswith(DEFAULT_IMG_EXTENSION):
                        fname = data['name']+DEFAULT_IMG_EXTENSION
                        
                    self._annotations[img_prefix+fname] = []
                    for img_frame in data['frames']:
                        self._annotations[img_prefix+fname].extend(img_frame['objects'])
        print(len(self._annotations))

In [5]:
example_set = Dataset( image_list = IMAGE_LIST_DIR, label_list = LABEL_LIST_DIR, data_format = Format.bdd)

70000
70000


In [6]:
fixed_coco = COCO(FIXED_COCO_ANNOTATIONS_FILE)

categories = fixed_coco.loadCats(fixed_coco.getCatIds())
category_names = [category['name'] for category in categories]
category_names.extend(['motor', 'rider'])
print('Custom COCO categories:\n{}\n'.format('\n'.join(category_names)))

loading annotations into memory...
Done (t=41.25s)
creating index...
index created!
Custom COCO categories:
person
bicycle
car
motorcycle
airplane
bus
train
truck
boat
traffic light
fire hydrant
stop sign
parking meter
bench
bird
cat
dog
horse
sheep
cow
elephant
bear
zebra
giraffe
backpack
umbrella
handbag
tie
suitcase
frisbee
skis
snowboard
sports ball
kite
baseball bat
baseball glove
skateboard
surfboard
tennis racket
bottle
wine glass
cup
fork
knife
spoon
bowl
banana
apple
sandwich
orange
broccoli
carrot
hot dog
pizza
donut
cake
chair
couch
potted plant
bed
dining table
toilet
tv
laptop
mouse
remote
keyboard
cell phone
microwave
oven
toaster
sink
refrigerator
book
clock
vase
scissors
teddy bear
hair drier
toothbrush
motor
rider



In [7]:
images, anns = [], []
img_offset, ann_index = 10000001, 100000000
num_imgs = len(example_set._annotations.keys())
    
for img_id, fname in enumerate(example_set._annotations.keys()):
    width, height = example_set._images[fname]['width'], example_set._images[fname]['height'] 
    
    if not fname.startswith(img_prefix):
        fname = img_prefix+fname
    dic = {'file_name': fname, 'id': img_offset+img_id, 'height': height, 'width': width}
    images.append(dic)
    
        
    # xy coords: [xstart, ystart, xstop, ystop] -> bbox = [x,y,width,height]
    for annotation in [x for x in example_set._annotations[fname] if x['category'] in category_names]:
        bbox = annotation['box2d']

        if bbox:
            # xy coords: [xstart, ystart, xstop, ystop] -> bbox = [x,y,width,height]
            xstart, ystart, xstop, ystop = float(bbox['x1']),float(bbox['y1']),float(bbox['x2']),float(bbox['y2'])


            if xstart < 0:
                xstart = 0.0
            if ystart < 0:
                ystart = 0.0
            if ystop <= 0:
                ystop = 3.0
            if xstop <= 0:
                xstop = 3.0

            # Get Points from Bounding Box
            pts = []
            pts.append((xstart , xstop))
            pts.append((xstop , ystart))
            pts.append((xstop , ystop))
            pts.append((xstart , ystop))


            # Convert XML Polygon pts to Coco-friendly bounding boxes
            points = MultiPoint(pts)
            xstart,ystart,xstop,ystop = points.bounds

            # Cast to Integers
            xstart,ystart,xstop,ystop = int(xstart),int(ystart),int(xstop),int(ystop)
            im = Image.open(example_set._images[fname]['coco_path'])
            binary_mask = np.zeros_like(im)[:,:,0]
            binary_mask[int(xstart):int(xstop), int(ystart):int(ystop)] +=1


            contours = measure.find_contours(binary_mask, 0.5, positive_orientation='low')
            polygons = []
            segmentations = []



            for contour in contours:
                poly = Polygon(contour)
                poly = poly.simplify(1.0, preserve_topology=False)

                if poly and poly.exterior:
                    polygons.append(poly)
                    segmentation = np.array(poly.exterior.coords).ravel().tolist()
                    segmentations.append(segmentation)

            # Combine the polygons to calculate the bounding box and area
            multi_poly = MultiPolygon(polygons)
            if multi_poly.bounds:
                x, y, max_x, max_y = multi_poly.bounds
                width = max_x - x
                height = max_y - y
                bbox = (x, y, width, height)
                area = multi_poly.area



                # Get Label
                if annotation['category'] == 'motorbike' or annotation['category'] == 'motor'  or annotation['category'] == 'rider':
                    category_id = fixed_coco.getCatIds(catNms=['motorcycle'])
                else:
                    category_id = fixed_coco.getCatIds(catNms=[annotation['category']])

                if not category_id: #Hardcode tv for now
                    category_id = fixed_coco.getCatIds(catNms=['tv'])

                if type(category_id) == list:
                    category_id = category_id[0]

                annotation = {
                    'segmentation': segmentations,
                    'iscrowd': 0,
                    'image_id': img_offset+img_id, # Don't want to conflict with existing dataset
                    'category_id': category_id,
                    'id': ann_index,
                    'bbox': bbox,
                    'area': area
                }
                ann_index+=1
                anns.append(annotation)
        

In [8]:
print(len(anns))

778155


In [9]:
from datetime import datetime

INFO = {
    "description": "Road Object-Detections Dataset based on MS COCO",
    "url": "https://kache.ai",
    "version": "0.0.1",
    "year": 2018,
    "contributor": "deanwebb",
    "date_created": datetime.utcnow().isoformat(' ')
}

LICENSES = [
    {
        "id": 1,
        "name": "The MIT License (MIT)",
        "url": "https://opensource.org/licenses/MIT",
        "description":  """
                        The MIT License (MIT)
                        Copyright (c) 2017 Matterport, Inc.

                        Permission is hereby granted, free of charge, to any person obtaining a copy
                        of this software and associated documentation files (the "Software"), to deal
                        in the Software without restriction, including without limitation the rights
                        to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
                        copies of the Software, and to permit persons to whom the Software is
                        furnished to do so, subject to the following conditions:

                        The above copyright notice and this permission notice shall be included in
                        all copies or substantial portions of the Software.

                        THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
                        IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
                        FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
                        AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
                        LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
                        OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
                        THE SOFTWARE.
                        """
    }
]

coco_output = {'info': INFO, 'licenses': LICENSES, 'images':images, 'annotations':anns, 'categories': categories}
with open(SCALABEL_COCO_ANNOTATIONS_FILE, 'w') as output_json_file:
    json.dump(coco_output, output_json_file)

In [14]:
testing_coco = COCO(SCALABEL_COCO_ANNOTATIONS_FILE)
category_ids = testing_coco.getCatIds(catNms=list(category_names))
image_ids = testing_coco.getImgIds()
image_data = testing_coco.loadImgs(image_ids[np.random.randint(0, len(image_ids))])[0]
print(image_data)

loading annotations into memory...
Done (t=6.56s)
creating index...
index created!
{'file_name': 'COCO_train2014_0000466dcc6e-4104e6c3.jpg', 'id': 10046272, 'height': 720, 'width': 1280}


In [11]:
# load and display instance annotations
image = io.imread(os.path.join(DATACACHE ,image_data['file_name']))
plt.imshow(image); plt.axis('off')
pylab.rcParams['figure.figsize'] = (128.0, 180.0)
annotation_ids = testing_coco.getAnnIds( catIds=category_ids, iscrowd=None)


annotations = testing_coco.loadAnns(annotation_ids)
print(len(annotations))


778155


In [13]:
print(annotations)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

